<a href="https://colab.research.google.com/github/Swagat-modder/DL-NLP-AI-projects/blob/main/Title_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku

In [2]:
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [3]:
df1 = pd.read_csv('/content/USvideos.csv')
df2 = pd.read_csv('/content/CAvideos.csv')
df3 = pd.read_csv('/content/GBvideos.csv')

#loading the datasets containing the category names
data1 = json.load(open('/content/US_category_id.json'))
data2 = json.load(open('/content/CA_category_id.json'))
data3 = json.load(open('/content/GB_category_id.json'))

In [4]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

In [5]:
#creating a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

In [6]:
#joining the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

In [7]:
#dropping rows based on duplicate videos
df = df.drop_duplicates('video_id')

In [8]:
#collecting only titles of entertainment videos
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()

In [9]:
#removing punctuations and converting text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

**Generating Sequences**

In [10]:
tokenizer = Tokenizer(num_words=5000)#using top 5000 words
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1

  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

  return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)

**Padding the sequences**

In [11]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len


predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

**Model Training using LSTM**

In [12]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()

  # Adding Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))

  # Adding Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))

  # Adding Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  return model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=20, verbose=5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


**Testing The Model**

In [14]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word
    return seed_text.title()

In [23]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len):
       for _ in range(next_words):
           token_list = tokenizer.texts_to_sequences([seed_text])[0]
           token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
           predicted_probs = model.predict(token_list, verbose=0)[0]  # Get probabilities

           # Filter predicted probabilities to only include top 5000 words
           predicted_probs[5000:] = 0  # Set probabilities of words outside top 5000 to 0

           predicted = np.argmax(predicted_probs)  # Get index of most likely word within top 5000

           output_word = ""
           for word, index in tokenizer.word_index.items():
               if index == predicted:
                   output_word = word
                   break
           # If output_word is empty (word not in top 5000), use a placeholder
           if not output_word:
              output_word = "<UNK>"  # Or any other placeholder you prefer

           seed_text += " " + output_word
       return seed_text.title()

In [26]:
print(generate_text("Bighero", 5, model, tokenizer, max_sequence_len)) # Added tokenizer as an argument

Bighero Episode 2 Ep One Urdu1
